# 틱텍토 기본 게임 형태

In [4]:
# 틱택토 구현
import random

# 게임 상태
class State:
    # 초기화 -----------------------------------------------------------------
    def __init__(self, pieces=None, enemy_pieces=None):
        # 돌 배치
        # 최초는 나의 돌 위치와 적의 돌 위치가 전부 0이다
        self.pieces       = pieces       if pieces       != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9

    # 돌의 수 취득 -----------------------------------------------------------
    # 나의 돌이던, 적의 돌이던 넣으면 1로 세팅된 개수를 획듯한다
    # 나의 돌개수와 적의 돌개수의 총합이 9면 더이상 넣을 돌이 없다
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count +=  1
        return count

    # 패배 여부 확인
    def is_lose(self):
        # 돌 3개 연결 여부
        # 클로저
        def is_comp(x, y, dx, dy):
          # 3X3이 게임판이므로, 
          # 3번의 스캔 : y방향으로
          for k in range(3):
            # 만약, 세로 스캔 범위를 넘어간 경우 => y가 0보다 작거나, 2보다 크거나 
            # 만약, 가로 스캔 범위를 넘어간 경우 => x가 0보다 작거나, 2보다 크거나 
            # 적의 돌 스캔 : self.enemy_pieces[x+y*3] x가 0,1,2일때, y가 0, 1, 2일대 값들중 0이 존재하면
            # 전부 1이 아니므로, 게임이 끝난것이 아님
            if y < 0 or 2 < y or x < 0 or 2 < x or self.enemy_pieces[x+y*3] == 0:
                return False
            # x,y는 dx, dy만큼 이동시킴 검사하기위해
            x, y = x+dx, y+dy
          return True

        # 2개 점검
        # 패배 여부 확인=> X자로 대각선 비교
        # (0 + 0*3), (1+1*3), (2+2*3) => 왼쭉상단부터 내려오는 대각선
        # (0 + 2*3), (1+1*3), (2+0*3) => 왼쪽하단부터 올라가는 대각선
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        
        # 6개 점검
        # 걸리면 패배 => 수평, 수직 비교
        for i in range(3):
          # (0, 0, 1, 0) or (0, 0, 0, 1)
          # (0 + 0*3), (1+0*3), (2+0*3) => 왼쪽상단부터 수평선
          # (0 + 0*3), (0+1*3), (0+2*3) => 왼쪽상단부터 수직선

          # (0, 1, 1, 0) or (1, 0, 0, 1)
          # (0 + 1*3), (1+1*3), (2+1*3) => 왼쪽상단부터 수직으로 두번째부터 수평선
          # (1 + 0*3), (1+1*3), (1+2*3) => 왼쪽상단부터 수평으로 두번째부터 수직선

          # (0, 2, 1, 0) or (2, 0, 0, 1)
          # (0 + 2*3), (1+2*3), (2+2*3) => 왼쪽상단부터 수직으로 세번째부터 수평선
          # (2 + 0*3), (2+1*3), (2+2*3) => 왼쪽상단부터 수평으로 세번째부터 수직선

          if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
              return True

        return False

    # 무승부 여부 확인 -------------------------------------------------------
    def is_draw(self):
      # 내가 놓을 돌수과 적이 놓을 돌수의 총합이 9면 무승부
      return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9

    # 게임 종료 여부 확인 ----------------------------------------------------
    def is_done(self):
      # 내가 졌거나, 내가 비겼으면 종료됨 ------------------------------------
      return self.is_lose() or self.is_draw()

    # 다음 상태 얻기 ---------------------------------------------------------
    def next(self, action):
      # 본인 돌을 카피 떠서
      pieces = self.pieces.copy()
      # 해당 칸에 돌을 넣고
      pieces[action] = 1
      # 적돌과, 내돌을 바꿔서 다시 생성 -> 둘다 랜덤으로 작동하므로 교차로 패를 선택하게 구동
      return State(self.enemy_pieces, pieces)

    # 합법적인 수의 리스트 얻기 ---------------------------------------------
    # 다음 수를 넣을수 있는 인덱스 위치 정보를 모아서 리턴 
    def legal_actions(self):
      # 비워있는 리스트
      actions = []
      # 9개의 칸을 구동
      for i in range(9):
        # 해당칸이 나도 0이고, 적도 0이라면
        if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
          # 해당 칸을 추가
          actions.append(i)
      # 돌을 넣을수 있는 해당 인덱스를 담은 리스트를 리턴
      return actions

    # 선 수 여부 확인 --------------------------------------------------------
    def is_first_player(self):
      # 내가둔수와, 적이 둔수가 동일하면 내차례 -> 게임 맨처음은 항상 내가 선수다
      # 다르면 적차례
      return self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)

    # 문자열 표시 ------------------------------------------------------------
    # 게임판 드로잉
    # 객체를 직접 찍으면 이 함수가 자동 호출된다
    def __str__(self):
      # 내가 선수면 : ('o', 'x'), 내가 후수면 ('x', 'o')
      # 나는 무조건 O, 적은 X 이니까
      # 돌을 선택하는 것을 같은 엔진에서 수핸하니까, 돌 놓는 순서에 따라 교차함
      ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
      # 빈 문자열
      str = ''
      # 9개의 방을 돌면서 
      for i in range(9):
        # 방을 1차 리스트로 폈음
        if self.pieces[i] == 1:         # 내 돌이 발견되면 '0'을 그림
            str += ox[0]
        elif self.enemy_pieces[i] == 1: # 적 돌이 발견되면 'x'을 그림
            str += ox[1]
        else:                           # 비워 있으면 '-' 표시
            str += '-'
        # 3번재 마다 줄바꿈
        if i % 3 == 2:
            str += '\n'
      return str

In [5]:
# 랜덤으로 행동 선택 ---------------------------------------------------------
# 현재 게임판을 넣으면
def random_action(state):
  # 돌을 넣을 수 있는 칸의 인덱스 정보를 획드
  legal_actions = state.legal_actions()
  # 그중에 위치를 램덤으로 구해서 해당 방의 인덱스를 리턴
  return legal_actions[random.randint(0, len(legal_actions)-1)]

In [6]:
# 랜덤과 랜덤의 대전

# 상태 생성
state = State()

# 게임 종료 시까지 반복
while True:
    # 게임 종료 시
    if state.is_done():
        break;

    # 행동 얻기
    # 돌을 넣을수 있는 위치값 횟득    
    action = random_action(state)

    # 다음 상태 얻기
    state = state.next(action)

    # 게임판 그리기
    print(state)
    print()

---
-o-
---


-x-
-o-
---


-xo
-o-
---


-xo
-o-
x--


-xo
-oo
x--


-xo
-oo
x-x


oxo
-oo
x-x


oxo
xoo
x-x


oxo
xoo
xox




In [7]:
-float('inf')

-inf

# 미니 맥스법을 활용한 상태 가치 계산법

- **자신에게 있어서 최선의 수를 선택하고, 상대는 최악의 수를 선택한는 가정에서 시작**
- 가장 좋은 수를 찾는 탐색 알고리즘
- 두사람이 대결하는 유한 확정 완전 정보 게임에서 많이 사용
  - 두사람 : 플레이어 2명
  - 대결 : 플레이어 사이에 의해가 완전한 대립(한명이 이익이면, 한명은 손해)
  - 유한 : 게임에서 둘수 있는 수의 숫자가 유한한 상태
  - 확정 : 주사위를 던지는것과 같은 무작위 요소가 없는 상태
  - 완전 정보 : 모든 정보가 두 플레이어 모두에게 공개된 상태
  - 바둑, 장기, 체스등이 해당

- 전략
  - 선수(자신)는 자신에게 가치가 높은 수를 선택
  - 후수(상대방)선수에 대해 가치가 낮은 수를 선택
  - 상태 가치값을의 부호를 반전해서, 선수던, 후수던 관계없이 상태 가치의 최댓값을 반환하도록 한다
    - 내수에서 값이 높던, 후수가 같이 높던 높은쪽을 찾는다 : 네가티브 맥스법
    - 단, 게임판이 크면 (장기, 체스)등 국면이 많이 경우 부적절함 => 몬테카를로/몬테카를로 트리로 해결

- 양의 무한대 : float('inf')
- 음의 무한대 : float('-inf')

In [8]:
float('inf'), float('-inf')

(inf, -inf)

In [13]:
# 미니맥스법을 활용한 상태 가치 계산
# 재귀적 호출로 계속해서 값을 세팅해 둔다 
# 패를 끝까지 둬보고 판단
def mini_max(state):
    # 그렇게 계속 돌면서 지면 패배 시, 상태 가치 -1
    if state.is_lose():
        return -1
    
    # 그렇게 계속 돌면서 비기면, 무승부 시, 상태 가치 0
    if state.is_draw():
        return  0

    # 합법적인 수의 상태 가치 계산
    # 음의 무한대
    #best_score = -float('inf')
    best_score = float('-inf')
    # 빈 곳을 찾아서, 돌을 넣을 위치를 구함
    for action in state.legal_actions():
        # 해당 돌을 넣어보고->그상태로 밑까지 파본다, 그렇게 해서 얻는 점수
        #tmp_score = mini_max(state.next(action))
        score = -mini_max(state.next(action))
        # 최고 점수보다 높으면, 점수 갱신
        if score > best_score:
            best_score = score
        #print( f'ts:{tmp_score} s:{score} bs:{best_score}' )
        #print('-'*20)
            
    # 합법적인 수의 상태 가치값 중 최대값 선택
    # 최대값 -> 1
    return best_score

# 미니맥스법을 활용한 행동 선택
def mini_max_action(state):
    # 합법적인 수의 상태 가치 계산
    # 최도 액션
    best_action = 0
    # 최고 점수
    #best_score  = -float('inf')
    best_score = float('-inf')
    # 문자열
    str = ['','']
    # 넣을수 있는 위치를 뒤져가면서
    for action in state.legal_actions():
      # 빈자리에 돌을 넣고 적과 나의 위치를 변경하고 게임 스티이트를 넣고
      # min, max법을 통해서 점수를 계산
      score = -mini_max(state.next(action))
      # 해당 점수가 현재 최고 점수보다 높으면
      if score > best_score:
        # 최고 점수를 받은 현재 돌의 위치
        best_action = action
        # 최고 점수를 받은 현재 점수
        best_score  = score
      
      # 해당 액션에 따른 점수 로그 출력
      # 해당 문자열에 계속 누적해서 기록
      str[0] = '{}{:2d},'.format(str[0], action)
      str[1] = '{}{:2d},'.format(str[1], score)
    # 최종 스코어 출력
    print('action:', str[0], '\nscore: ', str[1], '\n')

    # 합법적인 수의 상태 가치의 최대값을 가진 행동 반환
    # 가장 최고의 위치를 리턴
    return best_action

In [12]:
# 미니맥스법과 랜덤의 대전

# 상태 생성
state = State()

# 게임 종료 시까지 반복
while True:
    # 게임 종료 시
    if state.is_done():
        break

    # 행동 얻기 ------------------------------------------------------------------
    # 나의 행동은 미니맥스법
    if state.is_first_player():
        action = mini_max_action(state)
    # 적의 행동은 랜덤
    else:
        action = random_action(state)
    # --------------------------------------------------------------------------
    
    # 다음 상태 얻기
    state = state.next(action)

    # 문자열 표시
    print(state)
    print()

    #break

# 최고 가치가 높은 1인 후보들중에 맨 앞에 있는 돌의 위치를 선택
# mini_max앞에 음수가 붙어있고 그게 짝수번, 홀수번 발생하므로, +1 or -1이 나올수 있다

action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:   0, 0, 0, 0, 0, 0, 0, 0, 0, 

o--
---
---


o--
---
--x


action:  1, 2, 3, 4, 5, 6, 7, 
score:  -1, 1,-1, 0, 0, 1, 0, 

o-o
---
--x


o-o
-x-
--x


action:  1, 3, 5, 6, 7, 
score:   1, 1, 0, 1, 0, 

ooo
-x-
--x


